# From other Maosheng repo

In [1]:
import numpy as np 
import pickle
import torch

In [2]:
with open('pacific_data_old.pkl', 'rb') as f:
    laplacians, eigenvectors, eigenvalues, (b1, b2), _ = pickle.load(f)


eigvecs, eigvals = eigenvectors, eigenvalues
L1, L1_down, L1_up = laplacians
B1, B2 = b1, b2

In [3]:
total_var = []
total_div = []
total_curl = []
num_eigemodes = len(eigvals)
for i in range(num_eigemodes):
    total_var.append(eigvecs[:, i].T @ L1 @ eigvecs[:, i])
    total_div.append(eigvecs[:, i].T @ L1_down @ eigvecs[:, i])
    total_curl.append(eigvecs[:, i].T @ L1_up @ eigvecs[:, i])
    
grad_eflow = np.where(np.array(total_div) >= 1e-4)[0]
curl_eflow = np.where(np.array(total_curl) >= 1e-3)[0]
harm_eflow = np.where(np.array(total_var) < 1e-6)[0]

In [4]:
# tensor data type 
dtype = torch.float64

# convert the laplacians first to a list then to torch sparse csr tensors
L1 = torch.sparse_coo_tensor(L1.nonzero(), L1.data, L1.shape)
L1_down = torch.sparse_coo_tensor(L1_down.nonzero(), L1_down.data, L1_down.shape)
L1_up = torch.sparse_coo_tensor(L1_up.nonzero(), L1_up.data, L1_up.shape)

grad_evectors = torch.tensor(eigvecs[:, grad_eflow], dtype=dtype)
curl_evectors = torch.tensor(eigvecs[:, curl_eflow], dtype=dtype)
harm_evectors = torch.tensor(eigvecs[:, harm_eflow], dtype=dtype)
grad_evalues = torch.tensor(eigvals[grad_eflow], dtype=dtype)
curl_evalues = torch.tensor(eigvals[curl_eflow], dtype=dtype)
harm_evalues = torch.tensor(eigvals[harm_eflow], dtype=dtype)

/tmp/ipykernel_82285/1614225355.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  L1 = torch.sparse_coo_tensor(L1.nonzero(), L1.data, L1.shape)


In [5]:
print(f"Foudn {len(grad_eflow)} grad eigemodes, {len(curl_eflow)} curl eigemodes, {len(harm_eflow)} harm eigemodes")

Foudn 94 grad eigemodes, 406 curl eigemodes, 0 harm eigemodes


### Test against old eigenvectors

In [44]:
with open('./pacific_data1.pkl', 'rb') as f:
    _, old_eigenvectors, old_eigenvalues, _, _ = pickle.load(f)


assert np.allclose(eigenvectors, old_eigenvectors)
assert np.allclose(eigenvalues, old_eigenvalues)

### Save

In [64]:
grad_vecs_masked = grad_evectors.mT
curl_vecs_masked = curl_evectors.mT
harm_vecs_masked = harm_evectors.mT
grad_vals_masked = grad_evalues
curl_vals_masked = curl_evalues
harm_vals_masked = harm_evalues


torch.save({
    'grad_vecs': grad_vecs_masked,
    'curl_vecs': curl_vecs_masked,
    'harm_vecs': harm_vecs_masked,
    'grad_vals': grad_vals_masked,
    'curl_vals': curl_vals_masked,
    'harm_vals': harm_vals_masked
}, 'ocean_hodge_basis.pt')

# My reproduction
Just scipy.sparse.linalg.eigs the Hodge Laplacian and that's it

In [170]:
from topofm.distributions import Distribution


class EdgeGP(Distribution):
    """
    Zero-mean GP on edges of a 2-simplicial complex. 
    """
    def __init__(
        self, 
        grad_vecs: torch.Tensor,
        curl_vecs: torch.Tensor,
        harm_vecs: torch.Tensor,
        grad_vals: torch.Tensor,
        curl_vals: torch.Tensor,
        harm_vals: torch.Tensor,
        gp_type: str = 'diffusion',
        harm_sigma: float = 1.0, 
        grad_sigma: float = 1.0, 
        curl_sigma: float = 1.0, 
        harm_kappa: float = 1.0, 
        grad_kappa: float = 1.0, 
        curl_kappa: float = 1.0, 
    ):
        """
        Args: 
            harm_evals: [A]
            grad_evals: [B]
            curl_evals: [C]
            harm_evecs: [A, D]
            grad_evecs: [B, D]
            curl_evecs: [C, D]
            gp_type: 'diffusion'
        """
        super().__init__(validate_args=False)
        self.gp_type = gp_type

        # Harmonic forms 
        self.harm_variance = harm_sigma * torch.exp(- harm_kappa ** 2.0 / 2.0 * harm_vals) # [A]
        self.grad_variance = grad_sigma * torch.exp(- grad_kappa ** 2.0 / 2.0 * grad_vals) # [B]
        self.curl_variance = curl_sigma * torch.exp(- curl_kappa ** 2.0 / 2.0 * curl_vals) # [C]

        # Reshape eigenvalues and spectral variance
        self.spectral_variance = torch.concat([self.harm_variance, self.grad_variance, self.curl_variance], dim=0) # [A + B + C]
        self.eigenvectors = torch.concat([harm_vecs, grad_vecs, curl_vecs], dim=0) # [A + B + C, D]
        
        # Shapes 
        self._batch_shape = torch.Size()
        self._event_shape = self.spectral_variance.shape

    def sample(self, shape: torch.Size):
        return self.sample_spectral(shape)

    def sample_spectral(self, shape: torch.Size):
        """
        Sample spectral weights. 

        Args: 
            shape: [...]

        returns: [..., 3M]
        """
        epsilon = torch.randn(*shape, *self._event_shape) # [..., 3M]
        return self.spectral_variance.sqrt() * epsilon # [..., 3M]
        
    def sample_euclidean(self, shape: torch.Size):
        spectral_samples = self.sample_spectral(shape) # [..., 3M]
        return torch.einsum('md, ...m -> ...d', self.eigenvectors, spectral_samples)

In [171]:
payload = torch.load('ocean_hodge_basis.pt')

"""
initial_gp:
  type: 'diffusion'
  harm_sigma: 1e-5
  grad_sigma: 11.808
  curl_sigma: 12.6
  harm_kappa: 0.0
  grad_kappa: 10.36
  curl_kappa: 9.53

# Terminal GP
terminal_gp: 
  type: 'diffusion'
  harm_sigma: 1e-5
  grad_sigma: 1.0
  curl_sigma: 1e-5
  harm_kappa: 0.0
  grad_kappa: 10.0
  curl_kappa: 0.0
"""

init_gp_args = {
    'gp_type': 'diffusion',
    'harm_sigma': 1e-5,
    'grad_sigma': 11.808,
    'curl_sigma': 12.6,
    'harm_kappa': 0.0,
    'grad_kappa': 10.36,
    'curl_kappa': 9.53,
}

terminal_gp_args = {
    'gp_type': 'diffusion',
    'harm_sigma': 1e-5,
    'grad_sigma': 1.0,
    'curl_sigma': 1e-5,
    'harm_kappa': 0.0,
    'grad_kappa': 10.0,
    'curl_kappa': 0.0,
}

init_gp = EdgeGP(**payload, **init_gp_args)
terminal_gp = EdgeGP(**payload, **terminal_gp_args)

In [172]:
class HodgeBasis:
    def __init__(self, b1, b2, eigenbasis=None, laplacians=None):
        self.b1 = b1
        self.b2 = b2
        if laplacians is not None:
            self.L1, self.L1_down, self.L1_up = laplacians
        else:
            raise ValueError("Laplacians must be provided")
        if eigenbasis is not None:
            self.eig_evecs, self.eig_vals = eigenbasis
        else:
            raise ValueError("Eigenbasis must be provided")

        self.grad_idx, self.curl_idx, self.harm_idx = self.get_hodge_idx

    @property
    def get_hodge_idx(self):
        try:
            return self.grad_idx, self.curl_idx, self.harm_idx
        except AttributeError:
            total_var, total_div, total_curl = [], [], []
            for i in range(self.eig_vals.shape[0]):
                total_var.append(self.eig_evecs[:,i].T@self.L1@self.eig_evecs[:,i])
                total_div.append(self.eig_evecs[:,i].T@self.L1_down@self.eig_evecs[:,i])
                total_curl.append(self.eig_evecs[:,i].T@self.L1_up@self.eig_evecs[:,i]) 

            eps = 1e-6
            grad_idx = np.where(np.array(total_div) >= eps)[0]
            curl_idx = np.where(np.array(total_curl) >= eps)[0]
            harm_idx = np.where(np.array(total_var) < eps)[0]
            assert len(grad_idx) + len(curl_idx) + len(harm_idx) == self.eig_vals.shape[0]
            return grad_idx, curl_idx, harm_idx

    @property
    def get_hodge_basis(self):
        grad_evecs = self.eig_evecs[:, self.grad_idx]
        curl_evecs = self.eig_evecs[:, self.curl_idx]
        harm_evecs = self.eig_evecs[:, self.harm_idx]
        return harm_evecs, grad_evecs, curl_evecs 
    
    @property
    def get_hodge_eig_vals(self):
        grad_evals = self.eig_vals[self.grad_idx]
        curl_evals = self.eig_vals[self.curl_idx]
        harm_evals = self.eig_vals[self.harm_idx]
        return harm_evals, grad_evals, curl_evals



class OceanGP:
    def __init__(self, batch_size, eigen_basis, gp_params):
        self.harm_evals, self.grad_evals, self.curl_evals = eigen_basis.get_hodge_eig_vals
        self.harm_evecs, self.grad_evecs, self.curl_evecs = eigen_basis.get_hodge_basis
        self.batch_size = batch_size

        self.gp_type = gp_params['gp_type']
        self.mean = gp_params['mean']
        self.harm_sigma2 = gp_params['harm_sigma']
        self.grad_sigma2 = gp_params['grad_sigma']
        self.curl_sigma2 = gp_params['curl_sigma']
        self.harm_kappa = gp_params['harm_kappa']
        self.grad_kappa = gp_params['grad_kappa']
        self.curl_kappa = gp_params['curl_kappa']
        self.alpha = gp_params['alpha']
        if self.gp_type == 'matern':
               self.harm_nu = gp_params['harm_nu']
               self.grad_nu = gp_params['grad_nu']
               self.curl_nu = gp_params['curl_nu']   
               
        
    def get_cov_spectral(self):
        if self.gp_type == 'diffusion':
            harm_cov = self.harm_sigma2 *  np.exp(- self.harm_kappa**2/2 * self.harm_evals)
            grad_cov = self.grad_sigma2 *  np.exp(- self.grad_kappa**2/2 * self.grad_evals)
            curl_cov = self.curl_sigma2 *  np.exp(- self.curl_kappa**2/2 * self.curl_evals)
        elif self.gp_type == 'matern':
            harm_cov = self.harm_sigma2 * (2*self.hamr_nu/self.harm_kappa**2 + self.harm_evals)**(self.harm_nu/2)
            grad_cov = self.grad_sigma2 * (2*self.grad_nu/self.grad_kappa**2 + self.grad_evals)**(self.grad_nu/2)
            curl_cov = self.curl_sigma2 * (2*self.curl_nu/self.curl_kappa**2 + self.curl_evals)**(self.curl_nu/2)
            
        return np.concatenate([harm_cov, grad_cov, curl_cov])
        
    def sample(self, if_seed=True):
        n = self.batch_size
        cov_spectral = self.get_cov_spectral()
        hodge_evecs = np.concatenate([self.harm_evecs, self.grad_evecs, self.curl_evecs], axis=1)
        ocean_samples = ocean_sampler_gp(self.mean, cov_spectral, hodge_evecs, n, if_seed)
        return torch.Tensor(ocean_samples) 


def ocean_sampler_gp(mean, cov_spectral, hodge_evecs, sample_size, if_seed=False, seed=True):
    '''use this to sample from the GP for initial distribution '''
    n_spectrals = cov_spectral.shape[0]
    if if_seed:
        np.random.seed(seed)
        print("fix seed: {}".format(seed))
    v = np.random.multivariate_normal(np.zeros(n_spectrals), np.eye(n_spectrals), sample_size)
    assert v.shape == (sample_size,) + (n_spectrals,)
    ocean_flows = mean[:,None] + hodge_evecs @ (np.sqrt(cov_spectral)[:, None] * v.T)
    return ocean_flows.T

In [173]:
with open('pacific_data_old.pkl', 'rb') as f:
    laplacians, eigenvectors, eigenvalues, (b1, b2), _ = pickle.load(f)

eigvecs, eigvals = eigenvectors, eigenvalues
L1, L1_down, L1_up = laplacians
B1, B2 = b1, b2


mean = np.zeros(19023)

eigenbasis = HodgeBasis(
    b1=B1,
    b2=B2,
    laplacians=(L1, L1_down, L1_up),
    eigenbasis=[
        eigvecs,
        eigvals,
    ],
)



init_ocean_gp = OceanGP(
    batch_size=512,
    eigen_basis=eigenbasis,
    gp_params={
        'gp_type': 'diffusion',
        'mean': mean,
        'harm_sigma': 1e-5,
        'grad_sigma': 11.808,
        'curl_sigma': 12.6,
        'harm_kappa': 0.0,
        'grad_kappa': 10.36,
        'curl_kappa': 9.53,
        'alpha': 0.0,
        'harm_nu': 0.0,
        'grad_nu': 0.0,
        'curl_nu': 0.0,
    }
)


terminal_ocean_gp = OceanGP(
    batch_size=512,
    eigen_basis=eigenbasis,
    gp_params={
        'gp_type': 'diffusion',
        'mean': mean,
        'harm_sigma': 1e-5,
        'grad_sigma': 1.0,
        'curl_sigma': 1e-5,
        'harm_kappa': 0.0,
        'grad_kappa': 10.0,
        'curl_kappa': 0.0,
        'alpha': 0.0,
        'harm_nu': 0.0,
        'grad_nu': 0.0,
        'curl_nu': 0.0,
    }
)


assert torch.allclose(torch.as_tensor(init_ocean_gp.get_cov_spectral()), init_gp.spectral_variance)

In [174]:
batch_size = torch.Size([512])

sample_ocean = init_ocean_gp.sample()
sample_my = init_gp.sample(batch_size)
sample_my_euclidean = init_gp.sample_euclidean(batch_size)

sample_ocean.var(), sample_my.var(), sample_my_euclidean.var()

fix seed: True


(tensor(0.0067),
 tensor(0.2611, dtype=torch.float64),
 tensor(0.0067, dtype=torch.float64))

In [155]:
sample_ocean.mean(), sample_my.mean(), sample_my_euclidean.mean()

(tensor(1.1220e-05),
 tensor(-0.0002, dtype=torch.float64),
 tensor(2.2567e-06, dtype=torch.float64))

(tensor(0.0067),
 tensor(1.3584, dtype=torch.float64),
 tensor(0.0356, dtype=torch.float64))